In [390]:
# import the necessary libraries
from bs4 import BeautifulSoup
import requests
import json
import re
import datetime
import easyocr
import cv2
import geopy.distance

In [391]:
# this function handles the situation when the html is embedded within javascript
def javascript_html_parse(source_arg):
	"""Return the JSON object as a dictionary"""
	start = "<script>window.__PRELOADED_STATE__ = "
	end = "</script"
	x = source_arg[source_arg.find(start)+len(start):]
	x = x[:x.find(end)]
	x = json.loads(x)
	return x

In [394]:
# Obtain all of the URL's that need to be scraped from 
# This is the first URL we start at, here we are collecting all of the URL's to be scraped from into 
# local area URL's 
url = "https://www.rightmove.co.uk/house-prices-in-London.html"
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')
b = soup.find_all("a", class_="head")
london_urls = []
for i in b:
	london_urls.append(i["href"])

borough_urls = []
for url in london_urls:
	source = requests.get(url).text
	soup = BeautifulSoup(source, 'lxml')
	b = soup.find_all("a", class_="head")
	for i in b:
		borough_urls.append(i["href"])

local_area_urls = []
for url in borough_urls:
	source = requests.get(url).text
	soup = BeautifulSoup(source, 'lxml')
	b = soup.find_all("a", class_="head")
	for i in b:
		local_area_urls.append(i["href"])	

In [395]:
# iterate over all of the URL's and scrape from them, and store the scraped data into a variable called houses
m_values = ["square meters", "square metres", "square meter","square metre","square m","square mt","sqmt","sq mt","sq.mt", "sq. mt", "sqm", "sq m", "sq.m", "sq. m", "sq: m", "sq:m", "sq :m", "sq:  m", "sq : m", "meters2", "metres2", "meter2", "metre2", "mt2", "m2"]
f_values = ['square feet','square ft','square f','sqft','sq ft','sq.ft','sq. ft','sqf','sq f','sq.f','sq. f','sq: f','sq:f','sq :f','sq : f','sq :f','feet2','ft2','f2']
houses = {}
for area_url in local_area_urls[:5]:

	# while loop that obtains the address, property type, last sold price, last sold date, and its url (if it has one) from each house. 
	# Stores each value into a list. This is from the web page which shows different houses from a particular area.

	page_number = 1
	addresses, property_type, last_sold_price, last_sold_date, url = [], [], [], [], []
	# obtain the number of pages to use in the while loop condition
	source = requests.get(area_url).text
	a = javascript_html_parse(source)
	num_pages = a["pagination"]["last"]
    
	while page_number <= num_pages:

		if page_number == 1:
			source = requests.get(area_url).text
		else:
			source = requests.get(area_url+f"?page={page_number}").text

		# Scrape the address, property type, the price & date of the last sale, and the URL.
		x = javascript_html_parse(source)
		for house in x["results"]["properties"]:
			addresses.append(house["address"])
			property_type.append(house["propertyType"])
			last_sold_price.append(house["transactions"][0]["displayPrice"])
			last_sold_date.append(house["transactions"][0]["dateSold"])
			url.append(house["detailUrl"])



		page_number += 1


	# Store the scraped data for each property into the houses dictionary
	for i in range(len(addresses)):
		houses[addresses[i]] = {"property_type":property_type[i], "price":last_sold_price[i], "date":last_sold_date[i], "url": url[i]}


    # Now loop through all of the stored URL's for each property, and scrape more data from it.

	for v in houses.values():
		if v['url'] == '':
			continue
		else:
			source = requests.get(v['url']).text
			soup = BeautifulSoup(source, 'lxml')
			script = soup.find("script", {"type":"text/javascript"}).text
			# Regex used to convert the JSON data structure into a python dict 
			# Use regex to extract json data from the script text
			json_script = re.findall(("(?s)(?<=window.PAGE_MODEL = )(.*$)"), script)[0]
			# Transforming json data within string into dictionary
			json_dict = json.loads(json_script)

			# The bullet pointed features 
			try:
				features = json_dict["soldPropertyData"]["property"]["keyFeatures"]
			except TypeError: 
				pass
			else:
				v["features"] = features
			# The floorplan URL
			try:
				floorplan_url = json_dict["soldPropertyData"]["property"]["floorplans"][0]['url']
			except Exception:
				# seen a TypeError and an IndexError so I'm going to capture all exceptions raised
				pass
			else:
				v["floorplan_url"] = floorplan_url	
			# Scrape the pictures of the house ### commented this out for now, as I'm not using the image data
			#house_image_urls = []
			#for image in json_dict["soldPropertyData"]["property"]["images"]:
			#	house_image_urls.append(image['url'])
			#v["house_image_urls"] = house_image_urls
			# location of the house
			latitude = json_dict["soldPropertyData"]["property"]["location"]["latitude"]
			longitude = json_dict["soldPropertyData"]["property"]["location"]["longitude"]
			v["latitude"] = latitude
			v["longitude"] = longitude
			# proximity to the stations 
			station_names, distances = [], []
			for station in json_dict["soldPropertyData"]["property"]["nearestStations"]:
				station_names.append(station["name"])
				distances.append(station["distance"])
			station_proximities = {station:distance for station, distance in zip(station_names, distances)}
			v["station_proximities"] = station_proximities
			# size of the property
			units, sizes = [], []
			for i in json_dict["soldPropertyData"]["property"]["sizings"]:
				units.append(i['unit'])
				sizes.append(i['maximumSize'])
			dimensions = {unit:size for unit, size in zip(units, sizes)}
            #for label, size in dimensions.items():
             #   if label not in m_values and label not in f_values:
              #      del dimensions[label]

            # still haven't delt with the potential of having both a sqm and sqf, or just a sqf

			v["property_size"] = dimensions
            
            # JUST ABOVE HERE IS WHERE I COULD MAKE IT SO ONLY SQ FT IS OBTAINED - Still needs to be finished off

			# number of bedrooms and bathrooms
			bedrooms = json_dict["soldPropertyData"]["property"]['bedrooms']
			bathrooms = json_dict["soldPropertyData"]["property"]['bathrooms']
			v["bedrooms"] = bedrooms
			v["bathrooms"] = bathrooms

			# Whether the property is a new build or not
			new_build = json_dict["soldPropertyData"]["transactions"][0]['newBuild']
			v["new_build"] = new_build

In [396]:
# change price from a string into an int
def price_int(x):
    """Return an integer from a price string"""
    a = x[1:]
    b = a.replace(",", "")
    b = int(b)
    return b

# run the above function for all of the stored prices
for features in houses.values():
    y = features['price']
    z = price_int(y)
    features['price'] = z

In [397]:
# def size_floorplan_1(floorplan):
#     """Return the size data (in sq ft) from the floorplan"""
#     reader = easyocr.Reader(['en']) # GPU=false taken out for now
#     result = reader.readtext(floorplan)
    
#     scraped_text = ''
#     for i in result:
#         scraped_text += i[1]
#         scraped_text += ","
#     text_list = scraped_text.split(",")
    
#     m_patterns = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square meters|square metres|square meter|square metre|square m|square mt|sqmt|sq mt|sq.mt|sq. mt|sqm|sq m|sq.m|sq. m|sq: m|sq:m|sq :m|sq:  m|sq : m|meters2|metres2|meter2|metre2|mt2|m2)", re.IGNORECASE)
#     f_patterns = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square feet|square ft|square f|sqft|sq ft|sq.ft|sq. ft|sqf|sq f|sq: f|sq:f|sq :f|sq:  f|sq : f|sq.f|sq. f|feet2|ft2|f2)", re.IGNORECASE)
#     house_size = []
#     for x in text_list:
#         for match in re.finditer(m_patterns, x):
#             #print('%s : %s' % (match.groups()))
#             house_size.append(list(match.groups()))
#         for match in re.finditer(f_patterns, x):
#             #print('%s : %s' % (match.groups())) 
#             house_size.append(list(match.groups()))
#     for a in house_size:
#         try:
#             a[0] = int(a[0])
#         except ValueError:
#             a[0] = float(a[0])
            
#     # then here you could find the
#     house_size = dict((house_size))
#     print(house_size)
#     # Code to take the largest returned value from the floorplan
#     amount = 0
#     for size in house_size.keys():
#         if size > amount:
#             amount = size
#         largest_size = {}
#         largest_size[amount] = house_size[amount]
#         largest_size[amount] = largest_size[amount].lower()
        
        
#         # need to add values with colons, :
#         # alternatively, you could just remove the colon. This would also nehate this issue

#         metre_values = ["square meters", "square metres", "square meter", "square metre", "square m", "square mt", "sqmt", "sq mt",
#                 "sq.mt", "sq. mt", "sqm", "sq m", "sq.m", "sq. m", "meters2", "metres2", "meter2", "metre2", "mt2", "m2"]
#         feet_values = ["square feet","square ft","square f","sqft","sq ft","sq.ft","sq. ft","sqf","sq f","sq.f","sq. f","feet2",
#                "ft2","f2"]


#         if largest_size[amount] in metre_values:
#             # convert the size into square ft if sq m is scraped
#             del largest_size[amount]
#             largest_size[amount*10.764] = "sq ft"
#         elif largest_size[amount] in feet_values:
#             largest_size[amount] = "sq ft"
        
#         # it would be useful to add a litte conversion here, if the scraped data is in sq m, then multiply be 10.764,
        
#         return largest_size

In [398]:
def size_floorplan_2(floorplan):
    """Return the size data (in sq ft) from the floorplan"""
    reader = easyocr.Reader(['en']) # GPU=false taken out for now
    result = reader.readtext(floorplan)
    
    scraped_text = ''
    for i in result:
        scraped_text += i[1]
        scraped_text += ","
    text_list = scraped_text.split(",")
    
    m_patterns = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square meters|square metres|square meter|square metre|square m|square mt|sqmt|sq mt|sq.mt|sq. mt|sqm|sq m|sq.m|sq. m|sq: m|sq:m|sq :m|sq : m|sq :m|meters2|metres2|meter2|metre2|mt2|m2)", re.IGNORECASE)
    f_patterns = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square feet|square ft|square f|sqft|sq ft|sq.ft|sq. ft|sqf|sq f|sq.f|sq. f|sq: f|sq:f|sq :f|sq : f|sq :f|feet2|ft2|f2)", re.IGNORECASE)
    house_size = []
    for x in text_list:
        for match in re.finditer(m_patterns, x):
            #print('%s : %s' % (match.groups()))
            house_size.append(list(match.groups()))
        for match in re.finditer(f_patterns, x):
            #print('%s : %s' % (match.groups())) 
            house_size.append(list(match.groups()))
    for a in house_size:
        try:
            a[0] = int(a[0])
        except ValueError:
            a[0] = float(a[0])
            
    # then here you could find the
    house_size = dict((house_size))
    #print(house_size)
    house_size_swap = {}
    # code to swap keys and values
    for k, v in house_size.items():
        house_size_swap[v] = k

    house_size = house_size_swap
    
    # Code to take the largest returned value from the floorplan numbers
    amount = max(house_size.values())
    for i in house_size.keys():
        if house_size[i] == amount:
            size_label = i.lower()
    
    # Remove punctuation from the size labels       
    if "," in size_label:
        size_label = size_label.replace(",", "")
    if ":" in size_label:
        size_label = size_label.replace(":", "")
    if "." in size_label:
        size_label = size_label.replace(".", "")
    
    largest_size = {size_label:amount}
    
    
            
    # need to add values with colons, :
    # alternatively, you could just remove the colon. This would also nehate this issue

    metre_values = ["square meters", "square metres", "square meter", "square metre", "square m", "square mt", "sqmt", "sq mt", "sq mt", "sq mt", "sqm", "sq m", "sq m", "sq m", "meters2", "metres2", "meter2", "metre2", "mt2", "m2"]
    feet_values = ["square feet","square ft","square f","sqft","sq ft","sq  ft","sq. ft","sqf","sq f","sq f","sq f","feet2", "ft2","f2"]


    if size_label in metre_values:
        # convert the size into square ft if sq m is obtained
        largest_size["sq ft"] = amount*10.764
        del largest_size[size_label]
    elif size_label in feet_values:
        largest_size["sq ft"] = amount
        

        
    return largest_size["sq ft"]

In [399]:
# function that returns the sq foot value from the dictionaries that are scraped
def sq_foot_return(x):
    values_m = ["square meters", "sq meters", "square metres", "sq metres", "square meter", "sq meter", "square metre", "sq metre","square m", "square mt", "sqmt", "sq mt", "sq.mt", "sq. mt", "sqm", "sq m", "sq.m", "sq. m", "sq: m", "sq:m", "sq :m", "sq : m", "sq :m", "meters2", "metres2", "meter2", "metre2", "mt2", "m2"]
    values_f = ["square feet", "square ft", "square f", "sqft", "sq ft", "sq.ft", "sq. ft", "sqf", "sq f", "sq.f", "sq. f", "sq: f", "sq:f", "sq :f", "sq : f", "sq :f", "feet2", "ft2", "f2"]
    for k, v in x.items():
        if k.strip() in values_f:
            return x[k]
        if k.strip() in values_m:
            return x[k]*10.764
    return "Na"

In [400]:
di = {"ac": 22, "sq metres":23}
sq_foot_return(di)

247.57199999999997

In [401]:
size_floorplan_2('https://media.rightmove.co.uk/14k/13345/115507229/13345_31062102_FLP_00_0000.jpeg')

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\users\jacks\appdata\local\programs\python\python39\lib\site-packages\torchvision\models\_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will be removed in 0.15. Please access them via the appropriate Weights Enum instead.
  warnings.warn(
c:\users\jacks\appdata\local\programs\python\python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\users\jacks\appdata\local\programs\python\python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Progress: |██████████████████████████████████████████████████| 100.7% Complete

1303

In [402]:
# here is where the code should go to create this outstanding schools list
outstanding_schools = [[51.4974948, -0.1356583],
 [51.5202607, -0.0293396],
 [51.4935082, -0.1178424],
 [51.6569225, -0.1949252],
 [51.5202607, -0.0293396],
 [51.5345448, -0.2043853],
 [51.5202607, -0.0293396],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.6569225, -0.1949252],
 [51.5906113, -0.1109709],
 [51.5202607, -0.0293396],
 [51.538621, -0.1028346],
 [51.5202607, -0.0293396],
 [51.5672808, -0.2710568],
 [51.6569225, -0.1949252],
 [51.5132537, -0.3043136],
 [51.5255162, 0.0352163],
 [51.5202607, -0.0293396],
 [51.4609218, -0.373149],
 [51.6569225, -0.1949252],
 [51.4990805, -0.1938253],
 [51.5255162, 0.0352163],
 [51.5255162, 0.0352163],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.5202607, -0.0293396],
 [51.5436387, -0.0553621],
 [51.4935082, -0.1178424],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.5886383, -0.0117625],
 [51.5390261, -0.1425516],
 [51.5132537, -0.3043136],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.5886121, 0.0823982],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.5886383, -0.0117625],
 [51.5132537, -0.3043136],
 [51.5906113, -0.1109709],
 [51.458373, -0.1891356],
 [51.5202607, -0.0293396],
 [51.4990805, -0.1938253],
 [51.5202607, -0.0293396],
 [51.5672808, -0.2710568],
 [51.5886383, -0.0117625],
 [51.4990156, -0.22915],
 [51.5132537, -0.3043136],
 [51.5202607, -0.0293396],
 [51.4609218, -0.373149],
 [51.4974948, -0.1356583],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.4933675, 0.0098214],
 [51.5886121, 0.0823982],
 [51.5886121, 0.0823982],
 [51.502781, -0.087738],
 [51.4935082, -0.1178424],
 [51.5540666, 0.134017],
 [51.4935082, -0.1178424],
 [51.5886121, 0.0823982],
 [51.5886383, -0.0117625],
 [51.4611509, -0.0073177],
 [51.5390261, -0.1425516],
 [51.4609218, -0.373149],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5132537, -0.3043136],
 [51.4990805, -0.1938253],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.5886121, 0.0823982],
 [51.5672808, -0.2710568],
 [51.5436387, -0.0553621],
 [51.5886121, 0.0823982],
 [51.5202607, -0.0293396],
 [51.5390261, -0.1425516],
 [51.502781, -0.087738],
 [51.4609218, -0.373149],
 [51.5436387, -0.0553621],
 [51.4974948, -0.1356583],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.5123443, -0.0909852],
 [51.538621, -0.1028346],
 [51.5672808, -0.2710568],
 [51.5436387, -0.0553621],
 [51.5255162, 0.0352163],
 [51.5886383, -0.0117625],
 [51.538621, -0.1028346],
 [51.4935082, -0.1178424],
 [51.4990156, -0.22915],
 [51.538621, -0.1028346],
 [51.5906113, -0.1109709],
 [51.5906113, -0.1109709],
 [51.5906113, -0.1109709],
 [51.502781, -0.087738],
 [51.4935082, -0.1178424],
 [51.5202607, -0.0293396],
 [51.502781, -0.087738],
 [51.5202607, -0.0293396],
 [51.5540666, 0.134017],
 [51.4990156, -0.22915],
 [51.4609218, -0.373149],
 [51.4935082, -0.1178424],
 [51.5132537, -0.3043136],
 [51.6569225, -0.1949252],
 [51.4611509, -0.0073177],
 [51.4935082, -0.1178424],
 [51.538621, -0.1028346],
 [51.5202607, -0.0293396],
 [51.4933675, 0.0098214],
 [51.406025, 0.013156],
 [51.5886383, -0.0117625],
 [51.458373, -0.1891356],
 [51.5886383, -0.0117625],
 [51.502781, -0.087738],
 [51.4990805, -0.1938253],
 [51.4097742, -0.2108084],
 [51.4933675, 0.0098214],
 [51.6522994, -0.0807119],
 [51.5672808, -0.2710568],
 [51.502781, -0.087738],
 [51.4974948, -0.1356583],
 [51.580559, -0.341995],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.6569225, -0.1949252],
 [51.4611509, -0.0073177],
 [51.6522994, -0.0807119],
 [51.5132537, -0.3043136],
 [51.6522994, -0.0807119],
 [51.5132537, -0.3043136],
 [51.458373, -0.1891356],
 [51.4097742, -0.2108084],
 [51.6522994, -0.0807119],
 [51.5436387, -0.0553621],
 [51.4611509, -0.0073177],
 [51.502781, -0.087738],
 [51.6522994, -0.0807119],
 [51.4974948, -0.1356583],
 [51.458373, -0.1891356],
 [51.502781, -0.087738],
 [51.458373, -0.1891356],
 [51.5540666, 0.134017],
 [51.461311, -0.303742],
 [51.502781, -0.087738],
 [51.5672808, -0.2710568],
 [51.461311, -0.303742],
 [51.5906113, -0.1109709],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.458373, -0.1891356],
 [51.5886383, -0.0117625],
 [51.5886121, 0.0823982],
 [51.5672808, -0.2710568],
 [51.4609218, -0.373149],
 [51.4935082, -0.1178424],
 [51.577924, 0.2120829],
 [51.4990805, -0.1938253],
 [51.5886383, -0.0117625],
 [51.461311, -0.303742],
 [51.4935082, -0.1178424],
 [51.4990156, -0.22915],
 [51.5255162, 0.0352163],
 [51.502781, -0.087738],
 [51.5255162, 0.0352163],
 [51.5132537, -0.3043136],
 [51.5255162, 0.0352163],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.538621, -0.1028346],
 [51.3769529, -0.0956895],
 [51.6569225, -0.1949252],
 [51.580559, -0.341995],
 [51.5436387, -0.0553621],
 [51.5255162, 0.0352163],
 [51.4990805, -0.1938253],
 [51.439933, 0.154327],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.3769529, -0.0956895],
 [51.5436387, -0.0553621],
 [51.439933, 0.154327],
 [51.6569225, -0.1949252],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.4097742, -0.2108084],
 [51.6522994, -0.0807119],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.5886121, 0.0823982],
 [51.41233, -0.300689],
 [51.5436387, -0.0553621],
 [51.406025, 0.013156],
 [51.3769529, -0.0956895],
 [51.502781, -0.087738],
 [51.580559, -0.341995],
 [51.439933, 0.154327],
 [51.5906113, -0.1109709],
 [51.5351832, -0.4481378],
 [51.5255162, 0.0352163],
 [51.4974948, -0.1356583],
 [51.4097742, -0.2108084],
 [51.5436387, -0.0553621],
 [51.5202607, -0.0293396],
 [51.5390261, -0.1425516],
 [51.6569225, -0.1949252],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.4935082, -0.1178424],
 [51.5540666, 0.134017],
 [51.4974948, -0.1356583],
 [51.4933675, 0.0098214],
 [51.5540666, 0.134017],
 [51.3769529, -0.0956895],
 [51.5351832, -0.4481378],
 [51.4990156, -0.22915],
 [51.4935082, -0.1178424],
 [51.4611509, -0.0073177],
 [51.4611509, -0.0073177],
 [51.5886121, 0.0823982],
 [51.5906113, -0.1109709],
 [51.6522994, -0.0807119],
 [51.6522994, -0.0807119],
 [51.3769529, -0.0956895],
 [51.458373, -0.1891356],
 [51.4933675, 0.0098214],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.4097742, -0.2108084],
 [51.4935082, -0.1178424],
 [51.4611509, -0.0073177],
 [51.502781, -0.087738],
 [51.577924, 0.2120829],
 [51.406025, 0.013156],
 [51.5906113, -0.1109709],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.439933, 0.154327],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.5132537, -0.3043136],
 [51.4611509, -0.0073177],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.5202607, -0.0293396],
 [51.4935082, -0.1178424],
 [51.4611509, -0.0073177],
 [51.4990805, -0.1938253],
 [51.6522994, -0.0807119],
 [51.5886121, 0.0823982],
 [51.439933, 0.154327],
 [51.4935082, -0.1178424],
 [51.5906113, -0.1109709],
 [51.41233, -0.300689],
 [51.5906113, -0.1109709],
 [51.4611509, -0.0073177],
 [51.4611509, -0.0073177],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.4609218, -0.373149],
 [51.4611509, -0.0073177],
 [51.5906113, -0.1109709],
 [51.580559, -0.341995],
 [51.3769529, -0.0956895],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.5886121, 0.0823982],
 [51.461311, -0.303742],
 [51.4935082, -0.1178424],
 [51.5132537, -0.3043136],
 [51.6569225, -0.1949252],
 [51.4097742, -0.2108084],
 [51.5906113, -0.1109709],
 [51.6569225, -0.1949252],
 [51.5886383, -0.0117625],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.4609218, -0.373149],
 [51.577924, 0.2120829],
 [51.5351832, -0.4481378],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.5906113, -0.1109709],
 [51.5886121, 0.0823982],
 [51.5672808, -0.2710568],
 [51.439933, 0.154327],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.5886121, 0.0823982],
 [51.439933, 0.154327],
 [51.5351832, -0.4481378],
 [51.4609218, -0.373149],
 [51.5540666, 0.134017],
 [51.6569225, -0.1949252],
 [51.3769529, -0.0956895],
 [51.439933, 0.154327],
 [51.6569225, -0.1949252],
 [51.4609218, -0.373149],
 [51.4990805, -0.1938253],
 [51.3769529, -0.0956895],
 [51.580559, -0.341995],
 [51.4974948, -0.1356583],
 [51.3769529, -0.0956895],
 [51.4990156, -0.22915],
 [51.5351832, -0.4481378],
 [51.5202607, -0.0293396],
 [51.458373, -0.1891356],
 [51.461311, -0.303742],
 [51.5906113, -0.1109709],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.5132537, -0.3043136],
 [51.4609218, -0.373149],
 [51.5436387, -0.0553621],
 [51.5390261, -0.1425516],
 [51.5436387, -0.0553621],
 [51.406025, 0.013156],
 [51.502781, -0.087738],
 [51.458373, -0.1891356],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.461311, -0.303742],
 [51.5436387, -0.0553621],
 [51.5351832, -0.4481378],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.577924, 0.2120829],
 [51.3769529, -0.0956895],
 [51.4974948, -0.1356583],
 [51.5390261, -0.1425516],
 [51.458373, -0.1891356],
 [51.4974948, -0.1356583],
 [51.458373, -0.1891356],
 [51.5255162, 0.0352163],
 [51.4974948, -0.1356583],
 [51.6522994, -0.0807119],
 [51.4990156, -0.22915],
 [51.4990805, -0.1938253],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5202607, -0.0293396],
 [51.5886383, -0.0117625],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.5202607, -0.0293396],
 [51.5540666, 0.134017],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.4097742, -0.2108084],
 [51.5390261, -0.1425516],
 [51.4990156, -0.22915],
 [51.458373, -0.1891356],
 [51.5123443, -0.0909852],
 [51.461311, -0.303742],
 [51.458373, -0.1891356],
 [51.4097742, -0.2108084],
 [51.538621, -0.1028346],
 [51.5886121, 0.0823982],
 [51.4974948, -0.1356583],
 [51.458373, -0.1891356],
 [51.6569225, -0.1949252],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.6569225, -0.1949252],
 [51.4935082, -0.1178424],
 [51.580559, -0.341995],
 [51.4935082, -0.1178424],
 [51.538621, -0.1028346],
 [51.6522994, -0.0807119],
 [51.5672808, -0.2710568],
 [51.5255162, 0.0352163],
 [51.4990805, -0.1938253],
 [51.4974948, -0.1356583],
 [51.4990156, -0.22915],
 [51.5672808, -0.2710568],
 [51.6522994, -0.0807119],
 [51.538621, -0.1028346],
 [51.5906113, -0.1109709],
 [51.6522994, -0.0807119],
 [51.4974948, -0.1356583],
 [51.5132537, -0.3043136],
 [51.5202607, -0.0293396],
 [51.5132537, -0.3043136],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5351832, -0.4481378],
 [51.5390261, -0.1425516],
 [51.580559, -0.341995],
 [51.458373, -0.1891356],
 [51.538621, -0.1028346],
 [51.4935082, -0.1178424],
 [51.580559, -0.341995],
 [51.5351832, -0.4481378],
 [51.502781, -0.087738],
 [51.4990156, -0.22915],
 [51.5886121, 0.0823982],
 [51.6569225, -0.1949252],
 [51.4990805, -0.1938253],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.5132537, -0.3043136],
 [51.3769529, -0.0956895],
 [51.458373, -0.1891356],
 [51.5351832, -0.4481378],
 [51.4990805, -0.1938253],
 [51.502781, -0.087738],
 [51.458373, -0.1891356],
 [51.6522994, -0.0807119],
 [51.6569225, -0.1949252],
 [51.5672808, -0.2710568],
 [51.538621, -0.1028346],
 [51.5202607, -0.0293396],
 [51.4990805, -0.1938253],
 [51.6522994, -0.0807119],
 [51.4974948, -0.1356583],
 [51.4611509, -0.0073177],
 [51.580559, -0.341995],
 [51.577924, 0.2120829],
 [51.3769529, -0.0956895],
 [51.6569225, -0.1949252],
 [51.458373, -0.1891356],
 [51.502781, -0.087738],
 [51.41233, -0.300689],
 [51.5540666, 0.134017],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.5351832, -0.4481378],
 [51.6522994, -0.0807119],
 [51.5390261, -0.1425516],
 [51.5132537, -0.3043136],
 [51.502781, -0.087738],
 [51.5132537, -0.3043136],
 [51.5351832, -0.4481378],
 [51.5202607, -0.0293396],
 [51.4990805, -0.1938253],
 [51.5672808, -0.2710568],
 [51.3769529, -0.0956895],
 [51.4990805, -0.1938253],
 [51.4990156, -0.22915],
 [51.4974948, -0.1356583],
 [51.5886121, 0.0823982],
 [51.461311, -0.303742],
 [51.5351832, -0.4481378],
 [51.5390261, -0.1425516],
 [51.5255162, 0.0352163],
 [51.538621, -0.1028346],
 [51.4097742, -0.2108084],
 [51.5886121, 0.0823982],
 [51.3769529, -0.0956895],
 [51.5390261, -0.1425516],
 [51.577924, 0.2120829],
 [51.5886383, -0.0117625],
 [51.406025, 0.013156],
 [51.4990805, -0.1938253],
 [51.3769529, -0.0956895],
 [51.6522994, -0.0807119],
 [51.4935082, -0.1178424],
 [51.461311, -0.303742],
 [51.580559, -0.341995],
 [51.3769529, -0.0956895],
 [51.580559, -0.341995],
 [51.406025, 0.013156],
 [51.6522994, -0.0807119],
 [51.4990156, -0.22915],
 [51.4611509, -0.0073177],
 [51.4933675, 0.0098214],
 [51.4974948, -0.1356583],
 [51.6522994, -0.0807119],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.5886383, -0.0117625],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.4609218, -0.373149],
 [51.406025, 0.013156],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.5672808, -0.2710568],
 [51.5672808, -0.2710568],
 [51.4990805, -0.1938253],
 [51.5202607, -0.0293396],
 [51.5202607, -0.0293396],
 [51.580559, -0.341995],
 [51.5436387, -0.0553621],
 [51.538621, -0.1028346],
 [51.458373, -0.1891356],
 [51.458373, -0.1891356],
 [51.580559, -0.341995],
 [51.5390261, -0.1425516],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.5906113, -0.1109709],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.458373, -0.1891356],
 [51.461311, -0.303742],
 [51.5886383, -0.0117625],
 [51.5390261, -0.1425516],
 [51.458373, -0.1891356],
 [51.580559, -0.341995],
 [51.5255162, 0.0352163],
 [51.5906113, -0.1109709],
 [51.4990156, -0.22915],
 [51.4990805, -0.1938253],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.4611509, -0.0073177],
 [51.3769529, -0.0956895],
 [51.4933675, 0.0098214],
 [51.577924, 0.2120829],
 [51.6522994, -0.0807119],
 [51.458373, -0.1891356],
 [51.4933675, 0.0098214],
 [51.4611509, -0.0073177],
 [51.4933675, 0.0098214],
 [51.4609218, -0.373149],
 [51.406025, 0.013156],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.6522994, -0.0807119],
 [51.4933675, 0.0098214],
 [51.4974948, -0.1356583],
 [51.5906113, -0.1109709],
 [51.5351832, -0.4481378],
 [51.5436387, -0.0553621],
 [51.4990805, -0.1938253],
 [51.6569225, -0.1949252],
 [51.461311, -0.303742],
 [51.6569225, -0.1949252],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.4990805, -0.1938253],
 [51.5886121, 0.0823982],
 [51.5351832, -0.4481378],
 [51.6522994, -0.0807119],
 [51.5255162, 0.0352163],
 [51.461311, -0.303742],
 [51.4974948, -0.1356583],
 [51.502781, -0.087738],
 [51.5436387, -0.0553621],
 [51.5906113, -0.1109709],
 [51.580559, -0.341995],
 [51.5202607, -0.0293396],
 [51.5886121, 0.0823982],
 [51.439933, 0.154327],
 [51.5436387, -0.0553621],
 [51.458373, -0.1891356],
 [51.5255162, 0.0352163],
 [51.6569225, -0.1949252],
 [51.4933675, 0.0098214],
 [51.580559, -0.341995],
 [51.4974948, -0.1356583],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.4974948, -0.1356583],
 [51.502781, -0.087738],
 [51.5132537, -0.3043136],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.580559, -0.341995],
 [51.439933, 0.154327],
 [51.4611509, -0.0073177],
 [51.4933675, 0.0098214],
 [51.458373, -0.1891356],
 [51.4097742, -0.2108084],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.580559, -0.341995],
 [51.4935082, -0.1178424],
 [51.461311, -0.303742],
 [51.439933, 0.154327],
 [51.3769529, -0.0956895],
 [51.4935082, -0.1178424],
 [51.458373, -0.1891356],
 [51.439933, 0.154327],
 [51.580559, -0.341995],
 [51.4933675, 0.0098214],
 [51.461311, -0.303742],
 [51.3769529, -0.0956895],
 [51.577924, 0.2120829],
 [51.5436387, -0.0553621],
 [51.406025, 0.013156],
 [51.5132537, -0.3043136],
 [51.4974948, -0.1356583],
 [51.5886383, -0.0117625],
 [51.4990156, -0.22915],
 [51.406025, 0.013156],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.3769529, -0.0956895],
 [51.538621, -0.1028346],
 [51.5436387, -0.0553621],
 [51.538621, -0.1028346],
 [51.439933, 0.154327],
 [51.458373, -0.1891356],
 [51.3769529, -0.0956895],
 [51.5672808, -0.2710568],
 [51.4935082, -0.1178424],
 [51.41233, -0.300689],
 [51.577924, 0.2120829],
 [51.6569225, -0.1949252],
 [51.5202607, -0.0293396],
 [51.6569225, -0.1949252],
 [51.41233, -0.300689],
 [51.3769529, -0.0956895],
 [51.458373, -0.1891356],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.4609218, -0.373149],
 [51.5886121, 0.0823982],
 [51.458373, -0.1891356],
 [51.5132537, -0.3043136],
 [51.458373, -0.1891356],
 [51.4611509, -0.0073177],
 [51.461311, -0.303742],
 [51.4933675, 0.0098214],
 [51.4097742, -0.2108084],
 [51.5436387, -0.0553621],
 [51.577924, 0.2120829],
 [51.4990156, -0.22915],
 [51.406025, 0.013156],
 [51.580559, -0.341995],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.4990156, -0.22915],
 [51.458373, -0.1891356],
 [51.5123443, -0.0909852],
 [51.4990156, -0.22915],
 [51.5906113, -0.1109709],
 [51.3769529, -0.0956895],
 [51.580559, -0.341995],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.3769529, -0.0956895],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.5886121, 0.0823982],
 [51.4609218, -0.373149],
 [51.3769529, -0.0956895],
 [51.538621, -0.1028346],
 [51.6569225, -0.1949252],
 [51.5351832, -0.4481378],
 [51.6569225, -0.1949252],
 [51.4933675, 0.0098214],
 [51.458373, -0.1891356],
 [51.4935082, -0.1178424],
 [51.4974948, -0.1356583],
 [51.4609218, -0.373149],
 [51.6569225, -0.1949252],
 [51.406025, 0.013156],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.461311, -0.303742],
 [51.461311, -0.303742],
 [51.5351832, -0.4481378],
 [51.5540666, 0.134017],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.458373, -0.1891356],
 [51.5672808, -0.2710568],
 [51.458373, -0.1891356],
 [51.4974948, -0.1356583],
 [51.4974948, -0.1356583],
 [51.5906113, -0.1109709],
 [51.5906113, -0.1109709],
 [51.5540666, 0.134017],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5390261, -0.1425516],
 [51.4609218, -0.373149],
 [51.4990156, -0.22915],
 [51.6522994, -0.0807119],
 [51.5672808, -0.2710568],
 [51.5436387, -0.0553621],
 [51.502781, -0.087738],
 [51.461311, -0.303742],
 [51.5436387, -0.0553621],
 [51.5351832, -0.4481378],
 [51.5886121, 0.0823982],
 [51.580559, -0.341995],
 [51.4097742, -0.2108084],
 [51.5886383, -0.0117625],
 [51.4611509, -0.0073177],
 [51.577924, 0.2120829],
 [51.5132537, -0.3043136],
 [51.5390261, -0.1425516],
 [51.5132537, -0.3043136],
 [51.502781, -0.087738],
 [51.538621, -0.1028346],
 [51.4097742, -0.2108084],
 [51.5132537, -0.3043136],
 [51.461311, -0.303742],
 [51.5886383, -0.0117625],
 [51.6569225, -0.1949252],
 [51.41233, -0.300689],
 [51.577924, 0.2120829],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.458373, -0.1891356],
 [51.5202607, -0.0293396],
 [51.502781, -0.087738],
 [51.577924, 0.2120829],
 [51.5906113, -0.1109709],
 [51.4609218, -0.373149],
 [51.4609218, -0.373149],
 [51.5540666, 0.134017],
 [51.577924, 0.2120829],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.4933675, 0.0098214],
 [51.5886383, -0.0117625],
 [51.406025, 0.013156],
 [51.439933, 0.154327],
 [51.3769529, -0.0956895],
 [51.5202607, -0.0293396],
 [51.461311, -0.303742],
 [51.4974948, -0.1356583],
 [51.461311, -0.303742],
 [51.5886383, -0.0117625],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.458373, -0.1891356],
 [51.5886121, 0.0823982],
 [51.4097742, -0.2108084],
 [51.458373, -0.1891356],
 [51.458373, -0.1891356],
 [51.4611509, -0.0073177],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.4935082, -0.1178424],
 [51.406025, 0.013156],
 [51.458373, -0.1891356],
 [51.4990805, -0.1938253],
 [51.6569225, -0.1949252],
 [51.6569225, -0.1949252],
 [51.5672808, -0.2710568],
 [51.5132537, -0.3043136],
 [51.5886383, -0.0117625],
 [51.458373, -0.1891356],
 [51.3769529, -0.0956895],
 [51.5390261, -0.1425516],
 [51.4990156, -0.22915],
 [51.3769529, -0.0956895],
 [51.5886121, 0.0823982],
 [51.406025, 0.013156],
 [51.5202607, -0.0293396],
 [51.461311, -0.303742],
 [51.580559, -0.341995],
 [51.6569225, -0.1949252],
 [51.5390261, -0.1425516],
 [51.406025, 0.013156],
 [51.5886121, 0.0823982],
 [51.538621, -0.1028346],
 [51.461311, -0.303742],
 [51.461311, -0.303742],
 [51.5886121, 0.0823982],
 [51.6569225, -0.1949252],
 [51.580559, -0.341995],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.461311, -0.303742],
 [51.580559, -0.341995],
 [51.4990156, -0.22915],
 [51.5351832, -0.4481378],
 [51.5132537, -0.3043136],
 [51.5255162, 0.0352163],
 [51.5255162, 0.0352163],
 [51.538621, -0.1028346],
 [51.4609218, -0.373149],
 [51.5390261, -0.1425516],
 [51.5202607, -0.0293396],
 [51.580559, -0.341995],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.4990156, -0.22915],
 [51.4609218, -0.373149],
 [51.4935082, -0.1178424],
 [51.5390261, -0.1425516],
 [51.461311, -0.303742],
 [51.5132537, -0.3043136],
 [51.461311, -0.303742],
 [51.3769529, -0.0956895],
 [51.5540666, 0.134017],
 [51.4933675, 0.0098214],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.5906113, -0.1109709],
 [51.5255162, 0.0352163],
 [51.580559, -0.341995],
 [51.5886121, 0.0823982],
 [51.458373, -0.1891356],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.6569225, -0.1949252],
 [51.5672808, -0.2710568],
 [51.5672808, -0.2710568],
 [51.538621, -0.1028346],
 [51.5540666, 0.134017],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.5132537, -0.3043136],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.4933675, 0.0098214],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.461311, -0.303742],
 [51.538621, -0.1028346],
 [51.4990156, -0.22915],
 [51.4935082, -0.1178424],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.41233, -0.300689],
 [51.406025, 0.013156],
 [51.5351832, -0.4481378],
 [51.5132537, -0.3043136],
 [51.538621, -0.1028346],
 [51.41233, -0.300689],
 [51.538621, -0.1028346],
 [51.4990805, -0.1938253],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.4974948, -0.1356583],
 [51.4935082, -0.1178424],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.5255162, 0.0352163],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.4974948, -0.1356583],
 [51.538621, -0.1028346],
 [51.538621, -0.1028346],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.5202607, -0.0293396],
 [51.458373, -0.1891356],
 [51.4935082, -0.1178424],
 [51.4933675, 0.0098214],
 [51.3769529, -0.0956895],
 [51.4990156, -0.22915],
 [51.5390261, -0.1425516],
 [51.4097742, -0.2108084],
 [51.4990805, -0.1938253],
 [51.4097742, -0.2108084],
 [51.5202607, -0.0293396],
 [51.580559, -0.341995],
 [51.577924, 0.2120829],
 [51.4974948, -0.1356583],
 [51.406025, 0.013156],
 [51.5886121, 0.0823982],
 [51.5436387, -0.0553621],
 [51.5351832, -0.4481378],
 [51.580559, -0.341995],
 [51.4974948, -0.1356583],
 [51.502781, -0.087738],
 [51.4609218, -0.373149],
 [51.5255162, 0.0352163],
 [51.580559, -0.341995],
 [51.5886383, -0.0117625],
 [51.4990805, -0.1938253],
 [51.502781, -0.087738],
 [51.3769529, -0.0956895],
 [51.577924, 0.2120829],
 [51.577924, 0.2120829],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.538621, -0.1028346],
 [51.4609218, -0.373149],
 [51.406025, 0.013156],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.4990805, -0.1938253],
 [51.4935082, -0.1178424],
 [51.406025, 0.013156],
 [51.4933675, 0.0098214],
 [51.458373, -0.1891356],
 [51.6569225, -0.1949252],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.502781, -0.087738],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.5886121, 0.0823982],
 [51.577924, 0.2120829],
 [51.4611509, -0.0073177],
 [51.4611509, -0.0073177]]

In [403]:
def nearest_outstanding_school(lat, lon):
    """Return the KM distance to the nearest outstanding school"""
    coords_1 = (lat, lon)
    lower_boundary, upper_boundary = -0.05, 0.05
    school_distances = []
    for i in outstanding_schools:
        if (i[0] - lat > upper_boundary) or i[0] - lat < lower_boundary:
            continue
        elif (i[1] - lon > upper_boundary) or i[1] - lon < lower_boundary:
            continue
        else:
            coords_2 = (i[0], i[1])
            school_distances.append(geopy.distance.geodesic(coords_1, coords_2).km)

    return min(school_distances)

In [404]:
houses

{'Flat 131, Lauderdale Tower, Barbican, London, Greater London EC2Y 8BY': {'property_type': 'Flat',
  'price': 1500000,
  'date': '14 Apr 2022',
  'url': 'https://www.rightmove.co.uk/house-prices/details/england-111198221-14949232?s=6d33d105e453da910edf69d474e18a926d96b6c59d64c2d3aa33091244ad5ad5',
  'features': ['Type 1A',
   'Contemporary Kitchen',
   'Utility Room',
   'Wraparound Balcony',
   'Use Of Communal Gardens',
   'West Facing Views'],
  'floorplan_url': 'https://media.rightmove.co.uk/72k/71134/111198221/71134_30837396_FLP_00_0002.jpeg',
  'latitude': 51.519886,
  'longitude': -0.096743,
  'station_proximities': {'Barbican Station': 0.05088188581023095,
   'Moorgate Station': 0.3460585418348957,
   "St. Paul's Station": 0.35797745876363984},
  'property_size': {},
  'bedrooms': 3,
  'bathrooms': 2,
  'new_build': False},
 'Flat 27, Bayer House, Golden Lane Estate, London, Greater London EC1Y 0RN': {'property_type': 'Flat',
  'price': 660000,
  'date': '8 Apr 2022',
  'url':

In [405]:
# loop through all of the houses, if they don't have a property size but do have a floorplan, apply the OCR, and then return 
# # value (note a nonetype may be returned so you'll have to deal with that). 
# # If they have a don't have a fl

# # Loop through all of the houses, if they have their lon and lat available, return the distance to the nearest 
# # outstanding school
# for y in houses.values():
#     # Generate the feature of distance to the nearest outstanding school
#     if 'latitude' in y.keys() and 'longitude' in y.keys():
#         distance_km = nearest_outstanding_school(y['latitude'], y['longitude'])
#         y['nearest_outstanding_school'] = distance_km

    
#     # Attempt to generate the feature of the sq ft of the property
#     if 'property_size' in y.keys():
#         if (y['property_size'] == {}) and ('floorplan_url' in y.keys()):
#             try:
#                 ocr_size = size_floorplan_2(y['floorplan_url'])
#             except Exception:
#                 y['property_size'] = 'Na'
#             else:
#                 if ocr_size == None:
#                     y['property_size'] = 'Na'
#                 y['property_size'] = ocr_size
#     # If a property_size dictionary already exists, use it to find the sq ft of the property 
#     # code that handles when the property data has a dictionary with sqf and/or sq/f values in it
#     # Now you just need the f_values and m_values lists in the block of code
#         elif len(y['property_size']) >= 1:
#         # loop through the this dictionary
#         # if the dictionary contains a sq m value, set the value of property size to its sq f equivalent, 
#         # if the dictionary contains a sq f value, set it
#             for i, v in y['property_size'].items():
#                 p = i.strip().lower()
#                 if p in f_values:
#                     y['property_size'] = y['property_size'][v]
#                 elif p in m_values:
#                     y['property_size'] = y['property_size'][v]*10.764
                    
#         #elif (y['property_size'] == {}) and ('floorplan_url' not in y.keys()):
#          #   y['property_size'] = 'Na'
                
    
# # This still needs to be looked at
                
#     # if floorplan not in keys and property_size is {}
    
    
#     # This is where you could do an else statement, and set all of the prop_size dictionaries which end up in the data set 
#     # as ''
        

            
            
        
#  # need to handle for the situation when there is data in the property size dict, but its not related to sq sizes, 
# # and there is a floorplan URL
        
    
# #     else:
# #         if 'floorplan_url' in y.keys():
# #             ocr_size = size_floorplan_2(y['floorplan_url'])
# #             if ocr_size == None:
# #                 y['property_size'] = ''
# #             y['property_size'] = ocr_size
    
        

In [406]:
for y in houses.values():
    if 'property_size' in y.keys():
        if type(y['property_size']) == int or type(y['property_size']) == float:
            continue
        if (y['property_size'] == {} or type(y['property_size']) == str) and ('floorplan_url' in y.keys()):
            try:
                ocr_size = size_floorplan_2(y['floorplan_url'])
            except Exception:
                y['property_size'] = 'Na'
            else:
                if ocr_size == None or ocr_size == '':
                    y['property_size'] = 'Na'
                y['property_size'] = ocr_size
                # The line below this just sets the dictionary as the property size
                #y['property_size'] = ocr_size
                # This code actually sets the sq ft property size 
                ##for f in f_values:
                  ##  try:
                    ##    y['property_size'] = ocr_size[f]
                    ##except KeyError:
                      ##  pass
                    ##else:
                      ##  break
                            
        elif len(y['property_size']) >= 1 and type(y['property_size']) == dict:
            value = sq_foot_return(y['property_size'])
            y['property_size'] = value
            
            #for i, v in y['property_size'].items():
             #   p = i.strip().lower()
              #  if p in f_values:
               #     try:
                #        y['property_size'] = y['property_size'][i]
                 #   except KeyError:
                  #      pass
                   # else:
                    #    break
                #elif p in m_values:
                 #   try:
                  #      y['property_size'] = y['property_size'][i]*10.764
                   # except KeyError:
                    #    pass
                    #else:
                    # break
    if 'latitude' in y.keys() and 'longitude' in y.keys():
        distance_km = nearest_outstanding_school(y['latitude'], y['longitude'])
        y['nearest_outstanding_school'] = distance_km 
                
        
    

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |█████████████████████████████████████████████████████| 107.2% Complete

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |█████████████████████████████████████████████████████| 106.1% Complete

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


...

Progress: |████████████████████████████████████████████████████| 104.5% Complete

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |████████████████████████████████████████████████████| 105.0% Complete

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |████████████████████████████████████████████████████| 105.7% Complete

In [407]:
houses

{'Flat 131, Lauderdale Tower, Barbican, London, Greater London EC2Y 8BY': {'property_type': 'Flat',
  'price': 1500000,
  'date': '14 Apr 2022',
  'url': 'https://www.rightmove.co.uk/house-prices/details/england-111198221-14949232?s=6d33d105e453da910edf69d474e18a926d96b6c59d64c2d3aa33091244ad5ad5',
  'features': ['Type 1A',
   'Contemporary Kitchen',
   'Utility Room',
   'Wraparound Balcony',
   'Use Of Communal Gardens',
   'West Facing Views'],
  'floorplan_url': 'https://media.rightmove.co.uk/72k/71134/111198221/71134_30837396_FLP_00_0002.jpeg',
  'latitude': 51.519886,
  'longitude': -0.096743,
  'station_proximities': {'Barbican Station': 0.05088188581023095,
   'Moorgate Station': 0.3460585418348957,
   "St. Paul's Station": 0.35797745876363984},
  'property_size': 1087.9,
  'bedrooms': 3,
  'bathrooms': 2,
  'new_build': False,
  'nearest_outstanding_school': 0.9294055612957386},
 'Flat 27, Bayer House, Golden Lane Estate, London, Greater London EC1Y 0RN': {'property_type': 'Fl

In [408]:
# Need to convert the data into a quantitative format, which regression algorithms can learn/predict from
properties = {}
index = 0
# loop through all of the 
for h, i in houses.items():
    house_data = []
    if i['property_type'] == 'Flat':
        prop_type = 1
    elif i['property_type'] == 'Terraced':
        prop_type == 2
    elif i['property_type'] == 'Semi-Detached':
        prop_type = 3 
    elif i['property_type'] == 'Detached':
        prop_type = 4
    else:
        prop_type = 5
    house_data.append(prop_type)
    
    # Convert the data into the necessary string format
    date_obj = datetime.datetime.strptime(i['date'], "%d %b %Y")
    date_str = date_obj.strftime("%Y-%m-%d")
    house_data.append(date_str)
    
    if 'station_proximities' in i.keys() and i['station_proximities'] != {}:
        station_distance = min(i['station_proximities'].values())
        house_data.append(station_distance)
    else:
        house_data.append("NA")
        
    # You should have handled for the property_size difficulties by now
    if 'property_size' not in i.keys():
        house_data.append("NA")
    elif i['property_size'] == {} or i['property_size'] == 'Na' or i["property_size"] == None:
        house_data.append("NA")
    else:
        # This should return the sq foot of the property
        house_data.append(i["property_size"])
    
    # Add the bedrooms and bathrooms data   
    if "bedrooms" not in i.keys():
        house_data.append("NA")
    else:
        house_data.append(i["bedrooms"])
        
    if 'bathrooms' not in i.keys():
        house_data.append("NA")
    elif i['bathrooms'] == None:
        house_data.append("NA")
    else:
        house_data.append(i["bathrooms"])
        
    # New build
    if 'new_build' not in i.keys():
        house_data.append("NA")
    else:
        if i["new_build"]:
            house_data.append(1)
        else:
            house_data.append(0)
    
    if 'nearest_outstanding_school' not in i.keys():
        house_data.append("NA")
    else:
        house_data.append(i["nearest_outstanding_school"])
        
    house_data.append(i["price"])
    
    
        
    properties[h] = house_data
    index += 1
    
    

In [409]:
properties

{'Flat 131, Lauderdale Tower, Barbican, London, Greater London EC2Y 8BY': [1,
  '2022-04-14',
  0.05088188581023095,
  1087.9,
  3,
  2,
  0,
  0.9294055612957386,
  1500000],
 'Flat 27, Bayer House, Golden Lane Estate, London, Greater London EC1Y 0RN': [1,
  '2022-04-08',
  0.18723210717210148,
  618.8,
  2,
  1,
  0,
  1.18086626671716,
  660000],
 'Flat 192, Cromwell Tower, Barbican, London, Greater London EC2Y 8DD': [1,
  '2022-04-06',
  0.22041031108326287,
  1140.984,
  3,
  2,
  0,
  0.9197592059919366,
  1620000],
 'Flat 336, Willoughby House, Barbican, London, Greater London EC2Y 8BL': [1,
  '2022-04-01',
  0.11080892335977656,
  893.4119999999999,
  2,
  2,
  0,
  0.7979068234378649,
  1075000],
 '211, Bunyan Court, Barbican, London, Greater London EC2Y 8DH': [1,
  '2022-04-01',
  'NA',
  'NA',
  'NA',
  'NA',
  'NA',
  'NA',
  519500],
 '802, Frobisher Crescent, London, Greater London EC2Y 8HD': [1,
  '2022-03-28',
  0.17998907300614134,
  467,
  None,
  'NA',
  0,
  0.92812

In [410]:
properties['94, Bastable Avenue, Barking, Greater London IG11 0LF']

KeyError: '94, Bastable Avenue, Barking, Greater London IG11 0LF'

In [411]:
import pandas as pd
import numpy as np

In [412]:
properties_df = pd.DataFrame(properties)

In [413]:
properties_df

,"Flat 131, Lauderdale Tower, Barbican, London, Greater London EC2Y 8BY","Flat 27, Bayer House, Golden Lane Estate, London, Greater London EC1Y 0RN","Flat 192, Cromwell Tower, Barbican, London, Greater London EC2Y 8DD","Flat 336, Willoughby House, Barbican, London, Greater London EC2Y 8BL","211, Bunyan Court, Barbican, London, Greater London EC2Y 8DH","802, Frobisher Crescent, London, Greater London EC2Y 8HD","Flat 3, John Trundle Court, Barbican, London, Greater London EC2Y 8DJ","Flat 51, Breton House, Barbican, London, Greater London EC2Y 8DQ","Flat 246, Ben Jonson House, Barbican, London, Greater London EC2Y 8DL","Flat 16, Chequer Court, 3, Chequer Street, London, Greater London EC1Y 8PW",...,"Flat C, 25, Savage Gardens, London, Greater London EC3N 2AR","Third Floor Flat, 41, Crutched Friars, London, Greater London EC3N 2AE","Flat 308, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB","Flat 205, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB","Flat 304, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB","Part Of 7 Lovat Lane And Flat A, 5, Chy Hwel, London, Greater London EC3R 8DR","Flat 107, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB","Flat 108, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB","Flat 201, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB","Flat 207, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB"
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,2022-04-14,2022-04-08,2022-04-06,2022-04-01,2022-04-01,2022-03-28,2022-03-25,2022-03-25,2022-03-18,2022-03-08,...,1997-05-22,1997-03-07,1997-02-28,1997-02-14,1997-02-14,1997-01-30,1996-05-02,1996-05-02,1996-02-14,1996-01-29
2,0.050882,0.187232,0.22041,0.110809,NA,0.179989,0.051595,0.194288,0.171891,NA,...,NA,NA,0.096989,NA,NA,NA,0.084759,NA,NA,NA
3,1087.9,618.8,1140.984,893.412,NA,467,421.25,443,729,NA,...,NA,NA,520,NA,NA,NA,NA,NA,NA,NA
4,3,2,3,2,NA,None,None,None,1,NA,...,NA,NA,1,NA,NA,NA,1,NA,NA,NA
5,2,1,2,2,NA,NA,1,1,1,NA,...,NA,NA,NA,NA,NA,NA,1,NA,NA,NA
6,0,0,0,0,NA,0,0,0,0,NA,...,NA,NA,0,NA,NA,NA,0,NA,NA,NA
7,0.929406,1.180866,0.919759,0.797907,NA,0.928123,1.016726,1.01825,0.97384,NA,...,NA,NA,1.216211,NA,NA,NA,1.199453,NA,NA,NA
8,1500000,660000,1620000,1075000,519500,559000,475000,535000,670000,885000,...,155000,150000,85000,54000,50000,254000,70000,70000,44000,57850


In [414]:
properties_df = properties_df.T

In [415]:
properties_df

,0,1,2,3,4,5,6,7,8
"Flat 131, Lauderdale Tower, Barbican, London, Greater London EC2Y 8BY",1,2022-04-14,0.050882,1087.9,3,2,0,0.929406,1500000
"Flat 27, Bayer House, Golden Lane Estate, London, Greater London EC1Y 0RN",1,2022-04-08,0.187232,618.8,2,1,0,1.180866,660000
"Flat 192, Cromwell Tower, Barbican, London, Greater London EC2Y 8DD",1,2022-04-06,0.22041,1140.984,3,2,0,0.919759,1620000
"Flat 336, Willoughby House, Barbican, London, Greater London EC2Y 8BL",1,2022-04-01,0.110809,893.412,2,2,0,0.797907,1075000
"211, Bunyan Court, Barbican, London, Greater London EC2Y 8DH",1,2022-04-01,NA,NA,NA,NA,NA,NA,519500
...,...,...,...,...,...,...,...,...,...
"Part Of 7 Lovat Lane And Flat A, 5, Chy Hwel, London, Greater London EC3R 8DR",1,1997-01-30,NA,NA,NA,NA,NA,NA,254000
"Flat 107, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB",1,1996-05-02,0.084759,NA,1,1,0,1.199453,70000
"Flat 108, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB",1,1996-05-02,NA,NA,NA,NA,NA,NA,70000
"Flat 201, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB",1,1996-02-14,NA,NA,NA,NA,NA,NA,44000


In [416]:
properties_df.columns = ["Property Type", "Sold_Date", "Closest Station", "Size", "Bedrooms", "Bathrooms", "New Build", "Nearest OS", "Price"]

In [417]:
properties_df[0:25]

,Property Type,Sold_Date,Closest Station,Size,Bedrooms,Bathrooms,New Build,Nearest OS,Price
"Flat 131, Lauderdale Tower, Barbican, London, Greater London EC2Y 8BY",1,2022-04-14,0.050882,1087.9,3,2,0,0.929406,1500000
"Flat 27, Bayer House, Golden Lane Estate, London, Greater London EC1Y 0RN",1,2022-04-08,0.187232,618.8,2,1,0,1.180866,660000
"Flat 192, Cromwell Tower, Barbican, London, Greater London EC2Y 8DD",1,2022-04-06,0.22041,1140.984,3,2,0,0.919759,1620000
"Flat 336, Willoughby House, Barbican, London, Greater London EC2Y 8BL",1,2022-04-01,0.110809,893.412,2,2,0,0.797907,1075000
"211, Bunyan Court, Barbican, London, Greater London EC2Y 8DH",1,2022-04-01,NA,NA,NA,NA,NA,NA,519500
"802, Frobisher Crescent, London, Greater London EC2Y 8HD",1,2022-03-28,0.179989,467,None,NA,0,0.928123,559000
"Flat 3, John Trundle Court, Barbican, London, Greater London EC2Y 8DJ",1,2022-03-25,0.051595,421.25,None,1,0,1.016726,475000
"Flat 51, Breton House, Barbican, London, Greater London EC2Y 8DQ",1,2022-03-25,0.194288,443,None,1,0,1.01825,535000
"Flat 246, Ben Jonson House, Barbican, London, Greater London EC2Y 8DL",1,2022-03-18,0.171891,729,1,1,0,0.97384,670000
"Flat 16, Chequer Court, 3, Chequer Street, London, Greater London EC1Y 8PW",1,2022-03-08,NA,NA,NA,NA,NA,NA,885000


In [418]:
properties_df = properties_df.replace("NA", np.NaN)

In [419]:
properties_df

,Property Type,Sold_Date,Closest Station,Size,Bedrooms,Bathrooms,New Build,Nearest OS,Price
"Flat 131, Lauderdale Tower, Barbican, London, Greater London EC2Y 8BY",1,2022-04-14,0.050882,1087.900,3.0,2.0,0.0,0.929406,1500000
"Flat 27, Bayer House, Golden Lane Estate, London, Greater London EC1Y 0RN",1,2022-04-08,0.187232,618.800,2.0,1.0,0.0,1.180866,660000
"Flat 192, Cromwell Tower, Barbican, London, Greater London EC2Y 8DD",1,2022-04-06,0.220410,1140.984,3.0,2.0,0.0,0.919759,1620000
"Flat 336, Willoughby House, Barbican, London, Greater London EC2Y 8BL",1,2022-04-01,0.110809,893.412,2.0,2.0,0.0,0.797907,1075000
"211, Bunyan Court, Barbican, London, Greater London EC2Y 8DH",1,2022-04-01,NaN,NaN,NaN,NaN,NaN,NaN,519500
...,...,...,...,...,...,...,...,...,...
"Part Of 7 Lovat Lane And Flat A, 5, Chy Hwel, London, Greater London EC3R 8DR",1,1997-01-30,NaN,NaN,NaN,NaN,NaN,NaN,254000
"Flat 107, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB",1,1996-05-02,0.084759,NaN,1.0,1.0,0.0,1.199453,70000
"Flat 108, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB",1,1996-05-02,NaN,NaN,NaN,NaN,NaN,NaN,70000
"Flat 201, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB",1,1996-02-14,NaN,NaN,NaN,NaN,NaN,NaN,44000


In [420]:
# Here we've dropped all of the rows without all of the data 
prop_df = properties_df.dropna()

In [421]:
prop_df.Sold_Date.values

array(['2022-04-14', '2022-04-08', '2022-04-06', '2022-04-01',
       '2022-03-18', '2022-03-03', '2022-03-01', '2022-02-25',
       '2022-01-28', '2021-12-15', '2021-11-26', '2021-11-22',
       '2021-11-17', '2021-11-11', '2021-11-09', '2021-10-29',
       '2021-10-26', '2021-10-20', '2021-10-14', '2021-10-08',
       '2021-09-29', '2021-09-28', '2021-09-27', '2021-09-22',
       '2021-09-13', '2021-09-13', '2021-08-06', '2021-06-30',
       '2021-06-28', '2021-06-28', '2021-06-28', '2021-06-28',
       '2021-06-25', '2021-06-25', '2021-06-25', '2021-06-22',
       '2021-06-18', '2021-06-18', '2021-06-14', '2021-06-14',
       '2021-06-11', '2021-06-07', '2021-06-04', '2021-06-04',
       '2021-06-04', '2021-06-01', '2021-05-28', '2021-05-25',
       '2021-05-24', '2021-04-23', '2021-04-16', '2021-04-16',
       '2021-03-30', '2021-03-30', '2021-03-26', '2021-03-19',
       '2021-03-17', '2021-03-16', '2021-03-12', '2021-03-08',
       '2021-03-03', '2021-02-26', '2021-02-11', '2021-

In [422]:
# start = datetime.datetime(2020, 1, 1)
# end = datetime.datetime(2022, 7, 1)
# index = pd.date_range(start,end)
# index, len(index)


# multiplier = 5 
# l = list(range(0, len(index)*multiplier, multiplier))
# df = pd.DataFrame(l, index=index)
# df.index.name = 'date'
# df.columns = ['value']
#prop_df.loc[prop_df['Sold_Date'][2:4]=='22'] = 2
#prop_df.loc[prop_df['Sold_Date'][2:4]=='21'] = 1

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [423]:
prop_df

,Property Type,Sold_Date,Closest Station,Size,Bedrooms,Bathrooms,New Build,Nearest OS,Price
"Flat 131, Lauderdale Tower, Barbican, London, Greater London EC2Y 8BY",1,2022-04-14,0.050882,1087.900,3.0,2.0,0.0,0.929406,1500000
"Flat 27, Bayer House, Golden Lane Estate, London, Greater London EC1Y 0RN",1,2022-04-08,0.187232,618.800,2.0,1.0,0.0,1.180866,660000
"Flat 192, Cromwell Tower, Barbican, London, Greater London EC2Y 8DD",1,2022-04-06,0.220410,1140.984,3.0,2.0,0.0,0.919759,1620000
"Flat 336, Willoughby House, Barbican, London, Greater London EC2Y 8BL",1,2022-04-01,0.110809,893.412,2.0,2.0,0.0,0.797907,1075000
"Flat 246, Ben Jonson House, Barbican, London, Greater London EC2Y 8DL",1,2022-03-18,0.171891,729.000,1.0,1.0,0.0,0.973840,670000
...,...,...,...,...,...,...,...,...,...
"Flat G01, 28, Lovat Lane, London, Greater London EC3R 8EB",1,2015-02-02,0.119261,1820.000,2.0,2.0,1.0,0.560583,1900000
"Flat 510, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB",1,2006-06-05,0.104451,538.000,1.0,1.0,0.0,1.210958,282000
"7, Lovat Lane, London, Greater London EC3R 8DT",1,2005-11-03,0.102921,643.000,1.0,1.0,1.0,0.535692,236000
"Flat 210, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB",1,2000-03-31,0.104451,527.436,1.0,1.0,0.0,1.210958,130000


In [424]:
del prop_df['Sold_Date']

208

In [425]:
prop_df

,Property Type,Closest Station,Size,Bedrooms,Bathrooms,New Build,Nearest OS,Price
"Flat 131, Lauderdale Tower, Barbican, London, Greater London EC2Y 8BY",1,0.050882,1087.900,3.0,2.0,0.0,0.929406,1500000
"Flat 27, Bayer House, Golden Lane Estate, London, Greater London EC1Y 0RN",1,0.187232,618.800,2.0,1.0,0.0,1.180866,660000
"Flat 192, Cromwell Tower, Barbican, London, Greater London EC2Y 8DD",1,0.220410,1140.984,3.0,2.0,0.0,0.919759,1620000
"Flat 336, Willoughby House, Barbican, London, Greater London EC2Y 8BL",1,0.110809,893.412,2.0,2.0,0.0,0.797907,1075000
"Flat 246, Ben Jonson House, Barbican, London, Greater London EC2Y 8DL",1,0.171891,729.000,1.0,1.0,0.0,0.973840,670000
...,...,...,...,...,...,...,...,...
"Flat G01, 28, Lovat Lane, London, Greater London EC3R 8EB",1,0.119261,1820.000,2.0,2.0,1.0,0.560583,1900000
"Flat 510, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB",1,0.104451,538.000,1.0,1.0,0.0,1.210958,282000
"7, Lovat Lane, London, Greater London EC3R 8DT",1,0.102921,643.000,1.0,1.0,1.0,0.535692,236000
"Flat 210, Marlyn Lodge, Portsoken Street, London, Greater London E1 8RB",1,0.104451,527.436,1.0,1.0,0.0,1.210958,130000


In [426]:
x = prop_df.drop(['Price'], axis=1).values

In [427]:
y = prop_df['Price'].values

In [428]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [429]:
# split the data set into a training and testing set
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)

In [430]:
from sklearn.linear_model import LinearRegression
# Traing the model (on the training set)
ml = LinearRegression()
ml.fit(x_train,y_train)

LinearRegression()

In [431]:
# Predict the test set results
y_pred = ml.predict(x_test)
print(y_pred)

[ 939498.76633477 1121104.3032261  1684889.66719068  869545.12933069
  860471.6674529   700359.23178734  767402.48761359  386383.55680692
 1013900.3685108   590637.27530085  584010.76880101  597500.03181366
  690046.85869942 1695893.53513445  747493.24435579  997620.55315846
  368175.19521458 1520446.77566809 1113036.0051664  1589302.48979405
  851500.74054668  857228.37281613  896583.15796491  872200.16730826
 1201304.20931325  749597.4801095   767371.06908269  652986.16477155
  937312.07967362  404240.07170578  859991.23698727 1029870.44097263
 1693753.9403449   815277.54753964  587632.86510452 1467761.63388127
  832113.23943635  697029.14752812  786029.21623401  979887.92667359
  823285.58949235  578024.54773179 1555757.21945386  795915.90708821
  659833.12267841  787310.87128638  736561.73442948  893682.05817205
 1008910.34701929  776414.56887141 1018192.30701125  594958.13875041
 1570918.35214646  286613.2091838   997071.65646774  368247.00498065
  862781.06711357  735535.70661699

In [434]:
# evaluate the model 
from sklearn.metrics import r2_score, mean_absolute_error
r2_score(y_test, y_pred)

#mean_absolute_error(y_test, y_pred)
# 80 % score with the r score
# 55495 MAE (this will of)

133832.81297873743

In [ ]:
# Property type -> 
# Flat 1, Terranced 2, Semi-Detached 3, Detached 4
# Date (get it into YYYY-MM-DD format)
# Smallest station proximity
# property_size (in sqft), may need to do some conversions
# bedrooms
# bathrooms
# newbuild, 0 if false
# nearest outstanding school distance

# price (label)

In [ ]:
# Approach to help find the prices of nearby, similar properties
# THIS CODE NEEDS TO BE PERFORMED AFTER ALL OF THE HOUSES ARE STORED
# or even you could it after each URL iteration of the local area URL's, to save looping through all of the houses

######################## DO NOT RUN THIS CODE ###########################


for i, v in houses.items():
    local_similar_prices = []
    # need to store the postcode, each house has a postcode so this will be easier
    postcode = i[-8:]
    if postcode[0] == " ":
        postcode = i[-7:]
    postcode_area = postcode.split(" ")[0]
    prop_type = v["property_type"]
    
    for a, b in houses.items():
        postcode = i[-8:]
        if postcode[0] == " ":
            postcode = i[-7:]
        postcode_area1 = postcode.split(" ")[0]
        if postcode_area == postcode_area1:
            if b["property_type"] == prop_type:
                local_similar_prices.append(b['price'])
    
    avg_local_price = avg(local_similar_prices)
    v["avg_local_price"] = avg_local_price
    
    
####################################################################

In [ ]:
d = "hey,"

In [ ]:
d

In [ ]:
e = " hEy"

In [ ]:
e.strip().lower()